# 模型思路

 1. 將 content 當作第一句，answer 當作第二句，將兩者併在一起
 3. label 用 pre_y 的方法將目標人名做出 NER array，省略其他非目標人名
 4. 用 Bert+BiLSTM+CRF 模型，嘗試偵測 content 中 AML 人名

In [1]:
#https://blog.csdn.net/qq_42933419/article/details/102973424?ops_request_misc=&request_id=&biz_id=102&utm_term=BERT%20%E5%95%8F%E7%AD%94&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduweb~default-0-102973424
#https://blog.csdn.net/guangyacyb/article/details/105526482?ops_request_misc=&request_id=&biz_id=102&utm_term=BERT%20%E5%95%8F%E7%AD%94&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduweb~default-3-105526482
#https://mccormickml.com/2020/03/10/question-answering-with-a-fine-tuned-BERT/#part-1-how-bert-is-applied-to-question-answering

In [2]:
from keras.models import Model
from keras.layers import Input, Lambda, Bidirectional, LSTM, TimeDistributed, Dense
from keras.optimizers import Adam
from keras_bert import load_trained_model_from_checkpoint
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_accuracy
from keras_bert import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras_bert import AdamWarmup, calc_train_steps
import codecs
import time

Using TensorFlow backend.


In [3]:
from ckiptagger import WS, POS, NER

In [4]:
import os
import codecs
import pandas as pd
import numpy as np

bert_dir = r'C:\Users\rocker\Python - deep learning\Deep learning\keras 大神\bert'
config_path = os.path.join(bert_dir, 'bert_config.json')
checkpoint_path = os.path.join(bert_dir, 'bert_model.ckpt')
dict_path = os.path.join(bert_dir, 'vocab.txt')

In [5]:
data = pd.read_csv(r'C:\Users\rocker\Python - deep learning\Deep learning\keras 大神\content_df_4.csv')
data = data[(data["status"]=="ok")].drop(["url","context","raw_content","status"],axis =1)
#data = data[data['name'] != '[]']
data['name'] = data['name'].apply(lambda x: x.strip().replace('[','').replace(']','').replace('\'',''))
#data['aml_label'] = data['name'].apply(lambda x: 0 if x == '[]' else 1)

In [6]:
data = data.loc[~data['news_id'].isin([246, 1067, 1621, 1762, 2418, 2476, 2483, 2666, 2774, 3299,3144, 3676, 3898, 4064, 4597, 4659, 4950])]
#1762 3676

In [7]:
def tag_dict(data):
    y_token_dict = {}
    for tokens in data['name']:
        for token in tokens:
            if token not in y_token_dict.keys():
                y_token_dict[token] = len(y_token_dict)+1
    #word_dict_reverse = {v: k for k, v in y_token_dict.items()}
    #tag_tokenizer = Tokenizer(y_token_dict)
    
    return y_token_dict

def create_tokenizer(dict_path):
    token_dict = {}
    with codecs.open(dict_path, 'r', 'utf8') as reader:
        for line in reader:
            token = line.strip()
            token_dict[token] = len(token_dict)
    return token_dict

def transfer(i):
    if i != 0:
        return 1
    else:
        return 0

def encoded(tokenizer, data, question):
    
    x, y, z = [], [], []
    for content in data['content']:
        x1, x2 = tokenizer.encode(first=question, second=content, max_len=maxlen)
        x3 = [transfer(i) for i in x1]
        x.append(x1)
        y.append(x2)
        z.append(x3)
    return x, y, z

def tag_encoded(data, y_dict):
    z = []
    for tags in data['name']:
        x = []
        for tag in tags:
            x1 = y_dict.get(tag)
            x.append(x1)

        z.append(x)
    return z

In [49]:
maxlen = 400
batch_size = 8
epochs = 10
input_shape = (maxlen, )

In [50]:
question = '銀行為打擊犯罪訂定洗錢防制制度，哪些人是有涉及洗錢、詐騙等金融犯罪的高風險族群？'
token_dict = create_tokenizer(dict_path)
tokenizer = Tokenizer(token_dict)
input_id, segment_id, mask_input = encoded(tokenizer, data, question)

In [52]:
def pre_y(sentence, label, pad_num):
    y = np.zeros((len(sentence),pad_num))
    for i in range(len(sentence)):
        sen = sentence[i][0:pad_num]
        tokens = tokenizer.tokenize(first=sen)
        if len(tokens) > pad_num:
            tokens = tokens[0:pad_num-1]
            tokens.append('[SEP]')
        lab = label[i]
        lab_y = np.zeros(len(tokens))
        if lab!='':
            top = None
            #label只有一项
            if ',' not in lab:
                tokens_lab = tokenizer.tokenize(lab)
                num_lab=len(tokens_lab)-2
                for j in range(len(tokens)):
                    if j+num_lab-1<len(tokens):
                        if (tokens[j] == tokens_lab[1]) & (tokens[j + num_lab - 1] == tokens_lab[1 + num_lab - 1]):
                            top = j
                if top==None:
                    print('error:' + sen)
                    print(tokens,tokens_lab)
                    break
                lab_y[top] = 1  # B-COM
                for g in range(top + 1, top + num_lab):
                    lab_y[g] = 1  # I-COM
            #label有多项
            else:
                t=[]
                n=[]
                for u in lab.split(','):
                    tokens_lab = tokenizer.tokenize(u)
                    num_lab = len(tokens_lab) - 2
                    for j in range(len(tokens)):
                        if j + num_lab - 1 < len(tokens):
                            if (tokens[j] == tokens_lab[1]) & (tokens[j + num_lab - 1] == tokens_lab[1 + num_lab - 1]):
                                t.append(j)
                                n.append(num_lab)
                if len(t)==0:
                    print('error:' + sen)
                    break
                for t_num in range(len(t)):
                    lab_y[t[t_num]] = 1  # B-COM
                    for g in range(t[t_num]+1,t[t_num]+n[t_num]):
                        lab_y[g] = 1  # I-COM
            y[i]=np.lib.pad(lab_y, (0,pad_num-len(tokens)), 'constant', constant_values=(0,0))
        else:
            y[i][0] = 1    #若無答案則答案 = [CLS]

    return y #y.reshape((len(sentence),pad_num,1))

In [53]:
label = pre_y(list(data['content']), list(data['name']), maxlen)
label = label.reshape((label.shape[0], label.shape[1], 1))

In [54]:
def bert_BiLSTM_CRF_model():
    model = load_trained_model_from_checkpoint(config_path, checkpoint_path, training=True, seq_len=maxlen)
    bert_output = model.layers[-9].output
    X = Lambda(lambda x: x[:, 0: input_shape[0]])(bert_output)
    X = Bidirectional(LSTM(128, return_sequences=True))(X)
    output = Dense(1, activation='softmax')(X)
    model = Model(model.input, output)
    
    for layer in model.layers:
        layer.trainable = False
    model.layers[-1].trainable = True
    model.layers[-2].trainable = True
    return model

In [55]:
model = bert_BiLSTM_CRF_model()

In [56]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 400)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 400)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 400, 768), ( 16226304    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 400, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [57]:
total_steps, warmup_steps = calc_train_steps(
    num_example=data.shape[0],
    batch_size=batch_size,
    epochs=epochs,
    warmup_proportion=0.1,
)

optimizer = AdamWarmup(total_steps, warmup_steps, lr=1e-3, min_lr=1e-5)

In [58]:
model.compile(optimizer=optimizer, loss='mae')

In [59]:
model.fit([input_id, segment_id, mask_input],
          label,
          epochs=epochs,
          batch_size=batch_size,
          validation_split=0.1)

Train on 4387 samples, validate on 488 samples
Epoch 1/10
4387/4387 [==============================] - 282s 64ms/step - loss: 0.9960 - val_loss: 0.9962
Epoch 2/10
4387/4387 [==============================] - 284s 65ms/step - loss: 0.9960 - val_loss: 0.9962
Epoch 3/10
4387/4387 [==============================] - 277s 63ms/step - loss: 0.9960 - val_loss: 0.9962
Epoch 4/10
4387/4387 [==============================] - 277s 63ms/step - loss: 0.9960 - val_loss: 0.9962
Epoch 5/10
4387/4387 [==============================] - 281s 64ms/step - loss: 0.9960 - val_loss: 0.9962
Epoch 6/10
4387/4387 [==============================] - 282s 64ms/step - loss: 0.9960 - val_loss: 0.9962
Epoch 7/10
4387/4387 [==============================] - 284s 65ms/step - loss: 0.9960 - val_loss: 0.9962
Epoch 8/10
4387/4387 [==============================] - 284s 65ms/step - loss: 0.9960 - val_loss: 0.9962
Epoch 9/10
4387/4387 [==============================] - 281s 64ms/step - loss: 0.9960 - val_loss: 0.9962
Epoch 10

In [ ]:
y_orig = '今年8月14日，張安樂因遭控收受政治獻金卻未依法申報，還涉嫌協助兒子張瑋成立的華夏大地旅行社逃漏稅、掏空公司資產2千多萬元，遭台北地檢署依違反《政治獻金法》、幫助逃漏稅及業務侵占等罪起訴，但莊子逵是無辜的，莊子逵沒有犯罪。由於該旅行社有港幣近千萬元的境外資金疑透過地下匯兌匯入，張家對資金來源交代不清，檢調懷疑背後可能是中資。沒想到檢調追查意外發現，包括張安樂在內的統促黨幹部近年積極透過各種管道滲透地方宮廟，掌握宮廟資源以發展該黨組織，懷疑統促黨很可能利用政府對宗教團體採高度自由管理的巧門，藉部分宮廟財務不透明的有利條件，淪為中資入台洗錢管道。傳出統促黨總裁「白狼」透過各種管道滲透全台30地方宮廟，恐淪中資入台的洗錢管道（圖／翻攝畫面）檢視相片傳出統促黨總裁「白狼」透過各種管道滲透全台30地方宮廟，恐淪中資入台的洗錢管道（圖／翻攝畫面）更多根據＜鏡周刊＞報導，統促黨幹部主事的宮廟已遍及多個縣市，包括新北市、彰化縣、雲林縣、台南市等地都有類似案例。就連已有3百多年歷史的知名寺廟雲林北港朝天宮也成為統促黨拉攏目標，不但該廟董事吳東合入黨，統促黨乾隆黨部也聘任該廟董事蔡上元擔任榮譽顧問。其中，張安樂本身就擔任雲林斗六五路財神宮榮譽董事長，還成立中華伏羲協會擔任會長，並由統促黨主席張馥堂擔任理事長，每年都舉辦兩岸共祭人文始祖伏羲典禮等兩岸宗教交流活動，'
x1, x2 = tokenizer.encode(first=question, second=y_orig, max_len=maxlen)
x3 = [transfer(i) for i in x1]

In [ ]:
y_pred = np.argmax(model.predict([[x1], [x2], [x3]])[0], axis=-1)

In [ ]:
for i in range(maxlen):
    print(word_dict.get(x1[i]), y_pred[i])